### Module import

In [12]:
import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torchvision.utils import make_grid, save_image

from gradcam.gradcam import GradCAM, visualize_cam
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


### Load image

In [13]:
img_dir = 'images'
# img_name = 'collies.JPG'
# img_name = 'multiple_dogs.jpg'
# img_name = 'snake.JPEG'
img_name = 'water-bird.JPEG'
img_path = os.path.join(img_dir, img_name)

pil_img = PIL.Image.open(img_path)
img_path

'images\\water-bird.JPEG'

### preprocess image

In [14]:
torch_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])(pil_img).to(device)
normed_torch_img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(torch_img)[None]
torch_img.size()

torch.Size([3, 224, 224])

### Load torchvision models and make configs

In [15]:
#alexnet = models.alexnet(pretrained=True)
#vgg = models.vgg16(pretrained=True)
resnet = models.resnet101(pretrained=True)
#densenet = models.densenet161(pretrained=True)
#squeezenet = models.squeezenet1_1(pretrained=True)

In [16]:
normed_torch_img.size()

torch.Size([1, 3, 224, 224])

In [21]:
images = []
target_layers = ['layer4']
for layers, module in resnet.to(device).eval().named_modules():
    if layers in target_layers:
        gcam = GradCAM(resnet.to(device).eval(), module)
        print(gcam)
        mask, _ = gcam(normed_torch_img)
        print(normed_torch_img.size())
        heatmap, result = visualize_cam(mask, torch_img)        
        images.extend([torch_img.cpu(), heatmap, result])

len(images[2])
#grid_image = make_grid(images, nrow=5) 
#transforms.ToPILImage()(grid_image)

torch.Size([1, 3, 224, 224])


3

In [ ]:
configs = [
    #dict(model_type='alexnet', arch=alexnet, layer_name='features_11'),
    #dict(model_type='vgg', arch=vgg, layer_name='features_29'),
    dict(model_type='resnet', arch=resnet, layer_name='layer4'),
    #dict(model_type='densenet', arch=densenet, layer_name='features_norm5'),
    #dict(model_type='squeezenet', arch=squeezenet, layer_name='features_12_expand3x3_activation')
]


In [ ]:
for config in configs:
    config['arch'].to(device).eval()

cams = [[cls.from_config(**config) for cls in (GradCAM, GradCAMpp)]for config in configs]

### Feedforward image, calculate GradCAM/GradCAM++, and gather results

In [11]:
images = []
for gradcam, gradcam_pp in cams:
    mask, _ = gradcam(normed_torch_img)
    heatmap, result = visualize_cam(mask, torch_img)
    mask_pp, _ = gradcam_pp(normed_torch_img)
    heatmap_pp, result_pp = visualize_cam(mask_pp, torch_img)
    
    images.extend([torch_img.cpu(), heatmap, heatmap_pp, result, result_pp])
    
#grid_image = make_grid(images, nrow=5)
images

NameError: name 'cams' is not defined

### Show results 

In [ ]:
transforms.ToPILImage()(grid_image)